In [1]:
#for the matrix related operations
import numpy as np

from keras import layers
from keras.layers import Input,Dense,Activation,ZeroPadding2D,BatchNormalization,Flatten,Conv2D
from keras.layers import AveragePooling2D,MaxPooling2D,Dropout,GlobalMaxPooling2D,GlobalAveragePooling2D
from keras.utils import np_utils,print_summary

#for csv to dataframe
import pandas as pd
from keras.models import Sequential
from keras.callbacks import ModelCheckpoint
import keras.backend as K

Using TensorFlow backend.
C:\Users\super\AppData\Roaming\Python\Python37\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\super\AppData\Roaming\Python\Python37\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\super\AppData\Roaming\Python\Python37\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\super\AppData\Roa

# Loading Data and Normalizing

In [2]:
data=pd.read_csv("C:\\Users\\super\\Desktop\\Eckovation\\data.csv")
dataset=np.array(data)
np.random.shuffle(dataset)
X=dataset
Y=dataset
X=X[:,0:1024]
Y=Y[:,1024]

X_train=X[0:70000,:]
X_train=X_train/255.
X_test=X[70000:72001,:]
X_test=X_test/255.

#Reshape
Y=Y.reshape(Y.shape[0],1)
Y_train=Y[0:70000,:]
Y_train=Y_train.T
Y_test=Y[70000:72001,:]
Y_test=Y_test.T

# Printing the shape of Test and Train data

In [3]:
print("Number of training examples=" + str(X_train.shape[0]))
print("Number of test examples=" + str(X_test.shape[0]))
print("X_train shape=" + str(X_train.shape))
print("Y_train shape=" + str(Y_train.shape))
print("X_test shape=" + str(X_test.shape))
print("Y_test shape=" + str(Y_test.shape))

Number of training examples=70000
Number of test examples=2000
X_train shape=(70000, 1024)
Y_train shape=(1, 70000)
X_test shape=(2000, 1024)
Y_test shape=(1, 2000)


In [4]:
image_x=32
image_y=32

train_y=np_utils.to_categorical(Y_train)
test_y=np_utils.to_categorical(Y_test)
train_y=train_y.reshape(train_y.shape[1],train_y.shape[2])
test_y=test_y.reshape(test_y.shape[1],test_y.shape[2])
X_train=X_train.reshape(X_train.shape[0],image_x,image_y,1)
X_test=X_test.reshape(X_test.shape[0],image_x,image_y,1)

In [5]:
print("X_train shape=" + str(X_train.shape))
print("Y_train shape=" + str(train_y.shape))

X_train shape=(70000, 32, 32, 1)
Y_train shape=(70000, 37)


# Building the model

In [6]:
def keras_model(image_x,image_y):
    num_of_classes=37
    model=Sequential()
    model.add(Conv2D(filters=32,kernel_size=(5,5),input_shape=(image_x,image_y,1),activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2),padding='same'))
    model.add(Conv2D(64,(5,5),activation='relu'))
    model.add(MaxPooling2D(pool_size=(5,5),strides=(5,5),padding='same'))
    model.add(Flatten())
    model.add(Dense(num_of_classes,activation='softmax'))
    model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
    filepath="devanagari.h5"
    checkpoint1=ModelCheckpoint(filepath,monitor='val_acc',verbose=1,save_best_only=True,mode='max')
    callbacks_list=[checkpoint1]
    
    return  model,callbacks_list
                 

In [7]:
model,callbacks_list=keras_model(image_x,image_y)
model.fit(X_train,train_y,validation_data=[X_test,test_y],epochs=3,batch_size=64,callbacks=callbacks_list)
scores=model.evaluate(X_test,test_y,verbose=0)
print("CNN Error : %.2f%%" %(100-scores[1]*100))
print_summary(model)
model.save('devanagari.h5')


W0923 09:06:53.172819  6604 deprecation_wrapper.py:119] From C:\ProgramData\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:66: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0923 09:06:53.193065  6604 deprecation_wrapper.py:119] From C:\ProgramData\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:541: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0923 09:06:53.198420  6604 deprecation_wrapper.py:119] From C:\ProgramData\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:4432: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0923 09:06:53.224047  6604 deprecation_wrapper.py:119] From C:\ProgramData\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:4267: The name tf.nn.max_pool is deprecated. Please use tf.nn.max_pool2d instead.

W0923 09:06:53.299282  6604 deprecation_wrapper.py:119] From C:\ProgramData\Ana

Train on 70000 samples, validate on 2000 samples
Epoch 1/3
70000/70000 [==============================] - 120s 2ms/step - loss: 0.8020 - acc: 0.7724 - val_loss: 0.3450 - val_acc: 0.8980

Epoch 00001: val_acc improved from -inf to 0.89800, saving model to devanagari.h5
Epoch 2/3
70000/70000 [==============================] - 87s 1ms/step - loss: 0.2422 - acc: 0.9263 - val_loss: 0.2332 - val_acc: 0.9335

Epoch 00002: val_acc improved from 0.89800 to 0.93350, saving model to devanagari.h5
Epoch 3/3
70000/70000 [==============================] - 67s 951us/step - loss: 0.1583 - acc: 0.9529 - val_loss: 0.1837 - val_acc: 0.9495

Epoch 00003: val_acc improved from 0.93350 to 0.94950, saving model to devanagari.h5
CNN Error : 5.05%
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 28, 28, 32)        832       
_______________________________________________